In [1]:
from pathlib import Path
from tqdm import tqdm

#from kilosort.io import load_probe
from kilosort import run_kilosort, DEFAULT_SETTINGS

SAVE_PATH = '/ix1/pmayo/OneDrive/DATA/bins/kendra_scrappy_0097a.bin'

!pip install kilosort

In [2]:
import numpy as np

n_chan = 24
chanMap = np.arange(n_chan)
kcoords = np.zeros(n_chan)
xc = np.ones(n_chan)*5.5
yc = np.array([7.5 + 150*i for i in range(n_chan)])

probe = {
    'chanMap': chanMap,
    'xc': xc,
    'yc': yc,
    'kcoords': kcoords,
    'n_chan': n_chan
}

print(probe)

{'chanMap': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23]), 'xc': array([5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5,
       5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5]), 'yc': array([   7.5,  157.5,  307.5,  457.5,  607.5,  757.5,  907.5, 1057.5,
       1207.5, 1357.5, 1507.5, 1657.5, 1807.5, 1957.5, 2107.5, 2257.5,
       2407.5, 2557.5, 2707.5, 2857.5, 3007.5, 3157.5, 3307.5, 3457.5]), 'kcoords': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'n_chan': 24}


In [3]:
#probe = load_probe('/Users/kendranoneman/Projects/mayo/helperfunctions/mappings/probes/N19415.prb')

settings = DEFAULT_SETTINGS
settings['probe'] = probe
settings['n_chan_bin'] = 24
settings['fs'] = 30000
settings['filename'] = SAVE_PATH
settings['results_dir'] = '/ix1/pmayo/OneDrive/DATA/kilosort/kendra_scrappy_0097a'

In [4]:
ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = \
    run_kilosort(settings=settings, probe=probe)

kilosort.run_kilosort: Kilosort version 0.1.dev1164+gdfaaf6b.d20240502
kilosort.run_kilosort: Python version 3.10.14
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: System information:
kilosort.run_kilosort: Linux-3.10.0-1160.71.1.el7.x86_64-x86_64-with-glibc2.17 x86_64
kilosort.run_kilosort: x86_64
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using CUDA device: NVIDIA A100-PCIE-40GB 39.38GB
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Sorting /ix1/pmayo/OneDrive/DATA/bins/kendra_scrappy_0097a.bin
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 134174700
kilosort.run_kilos

In [6]:
from kilosort.io import save_preprocessing, load_ops

# NOTE: This will only create the .dat file, it will *NOT* update options for Phy.
#       If you want to use this with Phy, you will need to modify `params.py`
#       in the results directory to point to this new file. Additionally,
#       you must set `hp_filtered=True` and `dtype='int16'`.
ops_path = '/ix1/pmayo/OneDrive/DATA/kilosort/kendra_scrappy_0097a/ops.npy'
ops = load_ops(ops_path)
save_preprocessing(Path(ops_path).parent / 'temp_wh.dat', ops, bfile_path=SAVE_PATH)

kilosort.io :  
kilosort.io : ========================================
kilosort.io : Saving drift-corrected copy of data to: /ix1/pmayo/OneDrive/DATA/kilosort/kendra_scrappy_0097a/temp_wh.dat...
kilosort.io : Writing batch 0/2237...
kilosort.io : Writing batch 100/2237...
kilosort.io : Writing batch 200/2237...
kilosort.io : Writing batch 300/2237...
kilosort.io : Writing batch 400/2237...
kilosort.io : Writing batch 500/2237...
kilosort.io : Writing batch 600/2237...
kilosort.io : Writing batch 700/2237...
kilosort.io : Writing batch 800/2237...
kilosort.io : Writing batch 900/2237...
kilosort.io : Writing batch 1000/2237...
kilosort.io : Writing batch 1100/2237...
kilosort.io : Writing batch 1200/2237...
kilosort.io : Writing batch 1300/2237...
kilosort.io : Writing batch 1400/2237...
kilosort.io : Writing batch 1500/2237...
kilosort.io : Writing batch 1600/2237...
kilosort.io : Writing batch 1700/2237...
kilosort.io : Writing batch 1800/2237...
kilosort.io : Writing batch 1900/2237.

In [37]:
from pathlib import Path

import numpy as np
import os
os.environ["OMP_NUM_THREADS"] = '1'

print(os.environ['OMP_NUM_THREADS'])
import pandas as pd

from kilosort.io import load_ops

#print(os.environ['OMP_NUM_THREADS'])

# outputs saved to results_dir
results_dir = Path(settings['data_dir']).joinpath('kilosort4')
ops = load_ops(results_dir / 'ops.npy')
camps = pd.read_csv(results_dir / 'cluster_Amplitude.tsv', sep='\t')['Amplitude'].values
contam_pct = pd.read_csv(results_dir / 'cluster_ContamPct.tsv', sep='\t')['ContamPct'].values
chan_map =  np.load(results_dir / 'channel_map.npy')
templates =  np.load(results_dir / 'templates.npy')
chan_best = (templates**2).sum(axis=1).argmax(axis=-1)
chan_best = chan_map[chan_best]
amplitudes = np.load(results_dir / 'amplitudes.npy')
st = np.load(results_dir / 'spike_times.npy')
clu = np.load(results_dir / 'spike_clusters.npy')
firing_rates = np.unique(clu, return_counts=True)[1] * 30000 / st.max()
dshift = ops['dshift']

1


ModuleNotFoundError: No module named 'pandas'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec, rcParams
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
gray = .5 * np.ones(3)

fig = plt.figure(figsize=(10,10), dpi=100)
grid = gridspec.GridSpec(3, 3, figure=fig, hspace=0.5, wspace=0.5)

ax = fig.add_subplot(grid[0,0])
ax.plot(np.arange(0, ops['Nbatches'])*2, dshift);
ax.set_xlabel('time (sec.)')
ax.set_ylabel('drift (um)')

ax = fig.add_subplot(grid[0,1:])
t0 = 0
t1 = np.nonzero(st > ops['fs']*5)[0][0]
ax.scatter(st[t0:t1]/30000., chan_best[clu[t0:t1]], s=0.5, color='k', alpha=0.25)
ax.set_xlim([0, 5])
ax.set_ylim([chan_map.max(), 0])
ax.set_xlabel('time (sec.)')
ax.set_ylabel('channel')
ax.set_title('spikes from units')

ax = fig.add_subplot(grid[1,0])
nb=ax.hist(firing_rates, 20, color=gray)
ax.set_xlabel('firing rate (Hz)')
ax.set_ylabel('# of units')

ax = fig.add_subplot(grid[1,1])
nb=ax.hist(camps, 20, color=gray)
ax.set_xlabel('amplitude')
ax.set_ylabel('# of units')

ax = fig.add_subplot(grid[1,2])
nb=ax.hist(np.minimum(100, contam_pct), np.arange(0,105,5), color=gray)
ax.plot([10, 10], [0, nb[0].max()], 'k--')
ax.set_xlabel('% contamination')
ax.set_ylabel('# of units')
ax.set_title('< 10% = good units')

for k in range(2):
    ax = fig.add_subplot(grid[2,k])
    is_ref = contam_pct<10.
    ax.scatter(firing_rates[~is_ref], camps[~is_ref], s=3, color='r', label='mua', alpha=0.25)
    ax.scatter(firing_rates[is_ref], camps[is_ref], s=3, color='b', label='good', alpha=0.25)
    ax.set_ylabel('amplitude (a.u.)')
    ax.set_xlabel('firing rate (Hz)')
    ax.legend()
    if k==1:
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_title('loglog')